In [ ]:
pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"rizwanashaik1","key":"bb2915a41d549ccf9b699c36859e2764"}'}

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets list

ref                                                       title                                       size  lastUpdated          downloadCount  voteCount  usabilityRating  
--------------------------------------------------------  -----------------------------------------  -----  -------------------  -------------  ---------  ---------------  
nelgiriyewithana/apple-quality                            Apple Quality                              170KB  2024-01-11 14:31:07          10633        252  1.0              
shiivvvaam/top-youtuber-worldwide                         Top YouTubers Worldwide                     83KB  2024-02-05 09:33:23            699         37  1.0              
shiivvvaam/indian-districts-population-data               Indian Districts Population Data            46KB  2024-02-04 16:05:17            508         30  1.0              
hummaamqaasim/jobs-in-data                                Jobs and Salaries in Data Science           76KB  2023-12-25 20:03:32        

In [ ]:
! kaggle datasets download -d ahsanaseer/top-rated-tmdb-movies-10k

  0% 0.00/1.43M [00:00<?, ?B/s]
100% 1.43M/1.43M [00:00<00:00, 129MB/s]


In [ ]:
!unzip top-rated-tmdb-movies-10k.zip

Archive:  top-rated-tmdb-movies-10k.zip
  inflating: top10K-TMDB-movies.csv  


In [ ]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
movies_data = pd.read_csv('top10K-TMDB-movies.csv')

In [ ]:
# printing the first 5 rows of the dataframe
movies_data.head()

,id,title,genre,original_language,overview,popularity,release_date,vote_average,vote_count
0,278,The Shawshank Redemption,"Drama,Crime",en,Framed in the 1940s for the double murder of h...,94.075,1994-09-23,8.7,21862
1,19404,Dilwale Dulhania Le Jayenge,"Comedy,Drama,Romance",hi,"Raj is a rich, carefree, happy-go-lucky second...",25.408,1995-10-19,8.7,3731
2,238,The Godfather,"Drama,Crime",en,"Spanning the years 1945 to 1955, a chronicle o...",90.585,1972-03-14,8.7,16280
3,424,Schindler's List,"Drama,History,War",en,The true story of how businessman Oskar Schind...,44.761,1993-12-15,8.6,12959
4,240,The Godfather: Part II,"Drama,Crime",en,In the continuing saga of the Corleone crime f...,57.749,1974-12-20,8.6,9811


In [ ]:
# number of rows and columns in the data frame
movies_data.shape

(10000, 9)

In [ ]:
movies_data.describe()

,id,popularity,vote_average,vote_count
count,10000.000000,10000.000000,10000.000000,10000.000000
mean,161243.505000,34.697267,6.621150,1547.309400
std,211422.046043,211.684175,0.766231,2648.295789
min,5.000000,0.600000,4.600000,200.000000
25%,10127.750000,9.154750,6.100000,315.000000
50%,30002.500000,13.637500,6.600000,583.500000
75%,310133.500000,25.651250,7.200000,1460.000000
max,934761.000000,10436.917000,8.700000,31917.000000


In [ ]:
movies_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 10000 non-null  int64  
 1   title              10000 non-null  object 
 2   genre              9997 non-null   object 
 3   original_language  10000 non-null  object 
 4   overview           9987 non-null   object 
 5   popularity         10000 non-null  float64
 6   release_date       10000 non-null  object 
 7   vote_average       10000 non-null  float64
 8   vote_count         10000 non-null  int64  
dtypes: float64(2), int64(2), object(5)
memory usage: 703.2+ KB


In [ ]:
movies_data.columns

Index(['id', 'title', 'genre', 'original_language', 'overview', 'popularity',
       'release_date', 'vote_average', 'vote_count'],
      dtype='object')

In [ ]:
missing_values = movies_data.isnull().sum()
missing_values

id                    0
title                 0
genre                 3
original_language     0
overview             13
popularity            0
release_date          0
vote_average          0
vote_count            0
dtype: int64

In [ ]:
# Handling missing values in 'genre' and 'overview'
movies_data['genre'].fillna('Not Specified', inplace=True)  # Assuming 'Not Specified' is an appropriate placeholder for missing genres

# For 'overview', you can fill with an empty string or a placeholder value
movies_data['overview'].fillna('', inplace=True)

In [ ]:
# Check the count of missing values in each column
missing_values_count = movies_data.isnull().sum()
missing_values_count

id                   0
title                0
genre                0
original_language    0
overview             0
popularity           0
release_date         0
vote_average         0
vote_count           0
dtype: int64

In [ ]:
duplicate_rows = movies_data.duplicated()
duplicate_rows.sum()

0